In [285]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict

In [286]:
env = environment2.environment2()
user_list_2D = env.user_list_2D
user_list_3D = env.user_list_3D

In [287]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv'

In [288]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [289]:
fileNames2D=user_list_2D

In [290]:
fileNames2D

array(['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.csv',
       'data/NDSI-2D\\taskname_ndsi-2d-task_userid_954edb7c-

In [291]:
#subset of users
fileNames3D=['data/NDSI-3D\\taskname_ndsi-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv']

In [292]:
def get_ROISubset(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)
    replacement_dict = {(1, 2): 0, (3, 4): 1, (5, 6): 2, (7,8):3 , (9,10):4 , (11,12):5}
    combined_roi_subset = [next(v for k, v in replacement_dict.items() if i in k) for i in roi_subset]
    return combined_roi_subset



In [293]:
def get_Reward(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)
        # if i < len(data)/2:
        #     roi_subset.append(1)
        # else:
        #     roi_subset.append(2)
    return roi_subset

In [294]:
def get_ROISubset_3D(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
            if data['State'][i]=='Answering':
                    subset+=1
            if subset > 3:
                subset=3
            roi_subset.append(subset)
    return roi_subset

In [295]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None
    for i in range(len(dataframe)):
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i]="same"
      else:
        dataframe["high-level-action"][i]="change"
      current_state=dataframe["State"][i]

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])
      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Answeringsame'] + map['Answeringchange'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [296]:
from collections import defaultdict
def get_probabilities_region(dataframe):
    df=dataframe
    probabilities=[]
    map=defaultdict(lambda: defaultdict(int))
    # print(dataframe.head(5))
    actions=['Answering','Sensemaking','Foraging','Navigation']
    states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']
    for state in states:
      for action in actions:
        map[state][action]=1

    for i in range(len(df)):
      sum = 0
      map[df['Most_frequent_region'][i]][df['State'][i]] +=1
      for act in map[df['Most_frequent_region'][i]]:
                sum += map[df['Most_frequent_region'][i]][act]
      probs=map[df['Most_frequent_region'][i]][df['State'][i]]   / sum
      probabilities.append(probs)
    df['probabilities']= probabilities
    return df, map

In [297]:
!pip install pymannkendall
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk

You should consider upgrading via the 'C:\Users\Nischal Aryal\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [298]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [299]:
user=[]
ROI_Cycle=[]
SouthEastAnswering=[]
SouthEastSensemaking=[]
SouthEastForaging=[]
SouthEastNavigation=[]
SouthAnswering=[]
SouthSensemaking=[]
SouthForaging=[]
SouthNavigation=[]
OhioValleyAnswering=[]
OhioValleySensemaking=[]
OhioValleyForaging=[]
OhioValleyNavigation=[]
NorthernRockiesPlainsAnswering=[]
NorthernRockiesPlainsSensemaking=[]
NorthernRockiesPlainsForaging=[]
NorthernRockiesPlainsNavigation=[]
SouthWestAnswering=[]
SouthWestSensemaking=[]
SouthWestForaging=[]
SouthWestNavigation=[]
NorthWestAnswering=[]
NorthWestSensemaking=[]
NorthWestForaging=[]
NorthWestNavigation=[]
UpperMidwestAnswering=[]
UpperMidwestSensemaking=[]
UpperMidwestForaging=[]
UpperMidwestNavigation=[]
OtherAnswering=[]
OtherSensemaking=[]
OtherForaging=[]
OtherNavigation=[]
NoneAnswering=[]
NoneSensemaking=[]
NoneForaging=[]
NoneNavigation=[]

output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','SouthEastAnswering',
'SouthEastSensemaking',
'SouthEastForaging',
'SouthEastNavigation',
'SouthAnswering',
'SouthSensemaking',
'SouthForaging',
'SouthNavigation',
'OhioValleyAnswering',
'OhioValleySensemaking',
'OhioValleyForaging',
'OhioValleyNavigation',
'NorthernRockiesPlainsAnswering',
'NorthernRockiesPlainsSensemaking',
'NorthernRockiesPlainsForaging',
'NorthernRockiesPlainsNavigation',
'SouthWestAnswering',
'SouthWestSensemaking',
'SouthWestForaging',
'SouthWestNavigation',
'NorthWestAnswering',
'NorthWestSensemaking',
'NorthWestForaging',
'NorthWestNavigation',
'UpperMidwestAnswering',
'UpperMidwestSensemaking',
'UpperMidwestForaging',
'UpperMidwestNavigation',
'OtherAnswering',
'OtherSensemaking',
'OtherForaging',
'OtherNavigation',
'NoneAnswering',
'NoneSensemaking',
'NoneForaging',
'NoneNavigation'])


result_dataframe= pd.DataFrame(columns=['User','ROI_Cycle','SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None'])

for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities_region((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    mapper=dict()
    mapper['User']=u
    mapper['ROI_Cycle']=k
    print("Roi Cycle: #######################", k)
    for j in ['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']:
        #print("Region: ",j ,"Action Preferred: " ,max(mappings[j], key=mappings[j].get))
        maximum=max(mappings[j], key=mappings[j].get)
        if mappings[j][maximum] > 1 :
            action_preferred=maximum
        else:
            action_preferred='NA'
        mapper[j]=action_preferred
    some_df=pd.DataFrame([mapper])
    result_dataframe = result_dataframe.append(some_df, ignore_index=True)

        # series=(dfs.loc[dfs['Most_frequent_region'] == j])['probabilities']
        # if len(series) > 2 :
        #     sTest_state = StationarityTests()
        #     sTest_state.Mann_Kendall_Stationarity_Test(series.values, printResults = False)
        #     print("Region is: ", j, "  MKT Result is:  ", sTest_state.trend)
        # else :
        #      print("Region is: ", j, "  MKT Result is:  ","Not Enough Data")
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_Cycle.append(k)
    user.append(u)

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
Roi Cycle: ####################### 0
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

Roi Cycle: ####################### 2
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv
Roi Cycle: ####################### 0


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4
Roi Cycle: ####################### 5
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.csv


...

C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_da52acdd-3cea-483c-86e5-2c2740d371aa.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.csv
Roi Cycle: ####################### 0
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_20040\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)


In [300]:

result_dataframe.to_csv("data/NDSI-2D\\region_new-actions_roi_cycle.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [301]:
result_dataframe.head(20)

,User,ROI_Cycle,SouthEast,South,OhioValley,NorthernRockiesPlains,SouthWest,NorthWest,UpperMidwest,Northeast,West,Other,None
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,0,NA,Foraging,NA,Sensemaking,NA,NA,Navigation,NA,NA,NA,Foraging
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,0,NA,Foraging,NA,NA,NA,Navigation,NA,NA,Navigation,NA,Foraging
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,NA,Foraging,NA,Navigation,NA,Navigation,Foraging,Navigation,Navigation,NA,Foraging
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,NA,Navigation,NA,Foraging,Navigation,NA,NA,Navigation,NA,Sensemaking,Foraging
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,NA,Foraging,NA,Navigation,NA,NA,Navigation,NA,NA,NA,NA
5,userid_6d49fab8-273b-4a91-948b-ecd14556b049,0,NA,Foraging,NA,Navigation,NA,Navigation,NA,Navigation,Navigation,NA,Navigation
6,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,NA,Foraging,NA,Navigation,Navigation,NA,Navigation,Navigation,Navigation,Foraging,Foraging
7,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,NA,Navigation,NA,NA,Navigation,NA,Navigation,NA,NA,NA,Foraging
8,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,0,NA,Foraging,NA,Sensemaking,NA,Sensemaking,NA,NA,Navigation,NA,Navigation
9,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,1,NA,Foraging,NA,Navigation,NA,Navigation,Navigation,NA,Foraging,NA,Navigation


In [302]:
user=[]
ROI_Cycle=[]
Foragingchange=[]
Foragingsame=[]
Foraging=[]
Sensemakingchange=[]
Sensemakingsame=[]
Sensemaking=[]
Navigationsame=[]
Navigationchange=[]
Navigation=[]
AverageROIReward=[]
IdleTimeROI=[]
ForagingProportionROI=[]
NavigationProportionROI=[]


output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking','AverageROIReward','IdleTimeROI','ForagingProportionROI','NavigationProportionROI'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    print("Roi Cycle: #######################", k, " TimeSpent: ############## ", dfs['IdleTime'].sum())
    IdleTimeROI.append(dfs['IdleTime'].sum())
    AverageROIReward.append((df.loc[df['ROI_Cycle'] == k]['NDSI']*df.loc[df['ROI_Cycle'] == k]['ZoomLevel']).mean())

    for j in dfs.State.unique():
        series=(dfs.loc[(dfs['State'] == j) & (dfs["high-level-action"]=="same")])['probabilities']
        if len(series) > 2 :
            sTest_state = StationarityTests()
            sTest_state.Mann_Kendall_Stationarity_Test(series.values, printResults = False)
            print("State is: ", j, "  MKT Result is:  ", sTest_state.trend)
        else :
             print("State is: ", j, "  MKT Result is:  ","Not Enough Data")
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_Cycle.append(k)
    user.append(u)
  for k in range(len(maps)):
      #print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      #print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
      Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))


      #print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      #print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
      Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))


      #print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      #print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      ForagingProportionROI.append((maps[k]['Foragingsame']+ + maps[k]['Foragingchange'])/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']+maps[k]['Navigationsame'] + maps[k]['Navigationchange']+maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      NavigationProportionROI.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange'])/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']+maps[k]['Navigationsame'] + maps[k]['Navigationchange']+maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))

      print("##################################################")


output_dataframe['User']=user
output_dataframe['ROI_Cycle']=ROI_Cycle
output_dataframe['AverageROIReward']=AverageROIReward
output_dataframe['Foragingchange']=Foragingchange
output_dataframe['Foragingsame']=Foragingsame
output_dataframe['Foraging']=Foraging

output_dataframe['Sensemakingchange']=Sensemakingchange
output_dataframe['Sensemakingsame']=Sensemakingsame
output_dataframe['Sensemaking']=Sensemaking

output_dataframe['Navigationsame']=Navigationsame
output_dataframe['Navigationchange']=Navigationchange
output_dataframe['Navigation']=Navigation


output_dataframe['IdleTimeROI']=IdleTimeROI
output_dataframe['ForagingProportionROI']=ForagingProportionROI
output_dataframe['NavigationProportionROI']=NavigationProportionROI

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
Roi Cycle: ####################### 0  TimeSpent: ##############  175.86700000000002
State is:  Foraging   MKT Result is:   increasing
State is:  Navigation   MKT Result is:   increasing
State is:  Sensemaking   MKT Result is:   increasing
##################################################
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
Roi Cycle: ####################### 0  TimeSpent: ##############  74.52900000000001
State is:  Foraging   MKT Result is:   no trend
State is:  Navigation   MKT Result is:   no trend
State is:  Sensemaking   MKT Result is:   increasing
Roi Cycle: ####################### 1  TimeSpent: ##############  57.689000000000014
State is:  Navigation   MKT Result is:   increasing
State is:  Foraging   MKT Result is:   no trend
State is:  Sensemaking   MKT Result is:   Not Enough Data
Roi Cycle: #

In [303]:
output_dataframe.head(10)

,User,ROI_Cycle,Foragingchange,Foragingsame,Sensemakingchange,Sensemakingsame,Navigationsame,Navigationchange,Foraging,Navigation,Sensemaking,AverageROIReward,IdleTimeROI,ForagingProportionROI,NavigationProportionROI
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,0,0.363636,0.636364,0.142857,0.857143,0.777778,0.222222,11,18,14,1.175260,175.867,0.255814,0.418605
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,0,0.428571,0.571429,0.300000,0.700000,0.666667,0.333333,7,12,10,1.517471,74.529,0.241379,0.413793
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,0.428571,0.571429,0.750000,0.250000,0.750000,0.250000,7,20,4,1.407657,57.689,0.225806,0.645161
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,0.200000,0.800000,0.428571,0.571429,0.722222,0.277778,15,18,7,1.013608,44.531,0.375000,0.450000
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,0.666667,0.333333,0.500000,0.500000,0.666667,0.333333,3,6,2,1.076992,7.654,0.272727,0.545455
5,userid_6d49fab8-273b-4a91-948b-ecd14556b049,0,0.307692,0.692308,0.600000,0.400000,0.800000,0.200000,13,20,5,0.937663,93.881,0.342105,0.526316
6,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,0.272727,0.727273,0.600000,0.400000,0.821429,0.178571,11,28,5,1.008314,72.781,0.250000,0.636364
7,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,0.500000,0.500000,0.500000,0.500000,0.600000,0.400000,4,5,2,1.222001,23.268,0.363636,0.454545
8,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,0,0.333333,0.666667,0.428571,0.571429,0.714286,0.285714,9,14,7,0.614556,90.839,0.300000,0.466667
9,userid_72a8d170-77ae-400e-b2a5-de9e1d33a714,1,0.428571,0.571429,0.600000,0.400000,0.684211,0.315789,7,19,5,1.496508,54.856,0.225806,0.612903


In [304]:
output_dataframe.to_csv("data/NDSI-2D\\probability_distribution_roi_cycle.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [277]:
for u in output_dataframe['User'].unique()[1:]:
    print(u)
    user_df=(output_dataframe.loc[output_dataframe['User']==u]).sort_values(by=['ROI_Cycle'])
    print(user_df['Foragingsame'].values)
    sTest_state = StationarityTests()
    sTest_state.Mann_Kendall_Stationarity_Test(user_df['Foragingsame'].values, printResults = False)
    print(sTest_state.trend)
    print("#############################")

userid_44968286-f204-4ad6-a9b5-d95b38e97866
[0.66666667 0.33333333 0.6        0.5        0.33333333 0.85714286
 0.33333333]
no trend
#############################
userid_6d49fab8-273b-4a91-948b-ecd14556b049
[0.66666667 0.66666667 0.75       0.6        0.5       ]
no trend
#############################
userid_72a8d170-77ae-400e-b2a5-de9e1d33a714
[0.75       0.33333333 0.6        0.5        0.75       0.8       ]
no trend
#############################
userid_733a1ac5-0b01-485e-9b29-ac33932aa240
[0.33333333 0.5        0.77777778]
no trend
#############################
userid_7d014c75-453b-4188-9e4e-fb086984896a
[0.7826087  0.69230769 0.5        0.6        0.66666667 0.33333333
 0.5        0.5        0.5        0.6        0.66666667]
no trend
#############################
userid_82316e37-1117-4663-84b4-ddb6455c83b2
[0.33333333 0.8        0.33333333 0.5       ]
no trend
#############################
userid_8b544d24-3274-4bb0-9719-fd2bccc87b02
[0.6        0.5        0.72727273 0.25      ]
no

In [278]:
from scipy.stats import pearsonr
intented=0
for user in output_dataframe['User'].unique():

  test_df=(output_dataframe.loc[output_dataframe['User'] == user]).reset_index(drop=True)
  if len(test_df)>1:
      # calculate Pearson's correlation
      corr, _ = pearsonr(test_df['ROI_Cycle'], test_df['AverageROIReward'])
      if corr < -0.8 :
          intented+=1
      print("User ", user, " Intended")
      print('Pearsons correlation: %.3f' % corr)
print("Intended ", intented)

User  userid_44968286-f204-4ad6-a9b5-d95b38e97866  Intended
Pearsons correlation: -0.561
User  userid_6d49fab8-273b-4a91-948b-ecd14556b049  Intended
Pearsons correlation: 0.594
User  userid_72a8d170-77ae-400e-b2a5-de9e1d33a714  Intended
Pearsons correlation: 0.627
User  userid_733a1ac5-0b01-485e-9b29-ac33932aa240  Intended
Pearsons correlation: 0.925
User  userid_7d014c75-453b-4188-9e4e-fb086984896a  Intended
Pearsons correlation: 0.392
User  userid_82316e37-1117-4663-84b4-ddb6455c83b2  Intended
Pearsons correlation: 0.812
User  userid_8b544d24-3274-4bb0-9719-fd2bccc87b02  Intended
Pearsons correlation: 0.803
User  userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf  Intended
Pearsons correlation: 0.916
User  userid_93be0a02-3f7a-46c9-b526-aa956ed00856  Intended
Pearsons correlation: -0.060
User  userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d  Intended
Pearsons correlation: 0.031
User  userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8  Intended
Pearsons correlation: 0.935
User  userid_ac5b65d7-f260-453

In [ ]:
# user=[]
# ROI_Cycle=[]
# Foragingchange=[]
# Foragingsame=[]
# Foraging=[]
# Sensemakingchange=[]
# Sensemakingsame=[]
# Sensemaking=[]
# Navigationsame=[]
# Navigationchange=[]
# Navigation=[]
#
#
# actions=['Answering','Sensemaking','Foraging','Navigation']
# states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Other','None']
# for i in states:
#     for act in actions:
#           exec(f"{str(i)+str(act)} = {[]}")
# print(SouthEastAnswering)
# output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking'])
# for u in fileNames2D:
#   print("###########################",u)
#   index=[]
#
#   all_dfs=[]
#   maps=[]
#   all_dfs_region=[]
#   maps_region=[]
#   df = pd.read_csv(u)
#   u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
#   #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
#   u=u.rstrip('.csv')
#
#
#
#   #df['ROI_Cycle']=get_ROISubset(df)
#   df['ROI_Cycle']=get_ROISubset(df)
#
#   # print(df.ROI_Cycle.unique())
#   for k in df.ROI_Cycle.unique():
#     dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
#     dfs_region,mappings_region=get_probabilities_region((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
#     all_dfs.append(dfs)
#     all_dfs_region.append(dfs_region)
#     maps.append(mappings)
#     maps_region.append(mappings_region)
#     ROI_Cycle.append(k)
#     user.append(u)
#   for k in range(len(maps)):
#       print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
#       Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
#
#
#       print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
#       Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
#
#
#       print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
#       Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#       print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
#       Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#       Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#
#
#       print("##################################################")
#
#
#   for k in range(len(maps_region)):
#       actions=['Answering','Sensemaking','Foraging','Navigation']
#       states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Other','None']
#       for i in states:
#           sum=0
#           probabilities=[]
#           action=None
#           for act in mappings_region[i]:
#                     sum += mappings_region[i][act]
#                     probs=mappings_region[i][act]  / sum
#                     probabilities.append(probs)
#                     action=act
#
# output_dataframe['User']=user
# output_dataframe['ROI_Cycle']=ROI_Cycle
# output_dataframe['Foragingchange']=Foragingchange
# output_dataframe['Foragingsame']=Foragingsame
# output_dataframe['Foraging']=Foraging
#
# output_dataframe['Sensemakingchange']=Sensemakingchange
# output_dataframe['Sensemakingsame']=Sensemakingsame
# output_dataframe['Sensemaking']=Sensemaking
#
# output_dataframe['Navigationsame']=Navigationsame
# output_dataframe['Navigationchange']=Navigationchange
# output_dataframe['Navigation']=Navigation